In [ ]:
from google.colab import files

In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sachin099","key":"f6bd9b7688a5358dcddbe233aa83f959"}'}

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!mv /content/kaggle.json ~/.kaggle

mv: cannot stat '/content/kaggle.json': No such file or directory


In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!pip install kaggle

In [ ]:
!kaggle datasets download -d ananthr1/weather-prediction

Dataset URL: https://www.kaggle.com/datasets/ananthr1/weather-prediction
License(s): CC-BY-NC-SA-4.0
  0% 0.00/11.5k [00:00<?, ?B/s]
100% 11.5k/11.5k [00:00<00:00, 21.1MB/s]


In [ ]:
!unzip /content/weather-prediction.zip

Archive:  /content/weather-prediction.zip
  inflating: seattle-weather.csv     


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

In [ ]:
data= pd.read_csv("/content/seattle-weather.csv")
data.dropna(inplace=True)
data

,date,precipitation,temp_max,temp_min,wind,weather
0,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,2012-01-02,10.9,10.6,2.8,4.5,rain
2,2012-01-03,0.8,11.7,7.2,2.3,rain
3,2012-01-04,20.3,12.2,5.6,4.7,rain
4,2012-01-05,1.3,8.9,2.8,6.1,rain
...,...,...,...,...,...,...
1456,2015-12-27,8.6,4.4,1.7,2.9,rain
1457,2015-12-28,1.5,5.0,1.7,1.3,rain
1458,2015-12-29,0.0,7.2,0.6,2.6,fog
1459,2015-12-30,0.0,5.6,-1.0,3.4,sun


In [ ]:
data.duplicated().sum()

0

In [ ]:
data['date'] = pd.to_datetime(data['date'])

In [ ]:
training = data.iloc[:,2:3].values

In [ ]:
len(training)

1461

In [ ]:
def df_to_XY(df,window_size=10):
 X_train=[]
 y_train=[]

 for i in range(10,len(training)):
    X_train.append(training[i-10:i,0])
    y_train.append(training[i,0])

 X_train, y_train = np.array(X_train), np.array(y_train)
 return X_train, y_train

In [ ]:
WINDOW = 10
X,y = df_to_XY(data,WINDOW)
print(len(X),len(y))
X_train = X[:800]
y_train = y[:800]
X_val = X[800:1000]
y_val = y[800:1000]
X_test = X[1000:]
y_test = y[1000:]

1451 1451


In [ ]:
X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))
X_val = np.reshape(X_val,(X_val.shape[0],X_val.shape[1],1))
X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
model = Sequential()
#Addinf the first LSTM layer and some Dropout regularisation
model.add(LSTM(units=50, return_sequences = True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units=50))
model.add(Dropout(0.2))
#output layer
model.add(Dense(units=1))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.compile(optimizer='adam',loss='mean_squared_error')

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

In [ ]:
history=model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=100, batch_size=32)

Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 43ms/step - loss: 243.2188 - val_loss: 255.8708
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 102.9317 - val_loss: 162.9146
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 75.1159 - val_loss: 130.5502
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 64.7616 - val_loss: 112.9212
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 61.5137 - val_loss: 100.3270
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 55.0234 - val_loss: 91.9696
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 55.0429 - val_loss: 86.3920
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 58.5503 - val_loss: 82.2378
Epoch 9/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 51.0411 - val_loss: 79.8917
Epoch 10/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 53.0556 - val_loss: 78.7372
Epoch 11/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 54.1639 - val_loss: 76.9224
Epoch 12/100
25/25 ━━━━

In [ ]:
hist = pd.DataFrame(history.history)

In [ ]:
!pip install plotly
import plotly.express as px

history_loss = hist[['loss', 'val_loss']].reset_index()

# Now that px is imported, the following code should work:
fig = px.line(history_loss,
              x=history_loss.index,
              y=['loss', 'val_loss'],
              labels={'index': 'Epochs', 'value': 'Loss'},
              title='Loss & Val Loss')
fig.update_layout(width=700, height=500)

fig.show()

In [ ]:
train_pred = model.predict(X_train).flatten()
val_pred = model.predict(X_val).flatten()
test_pred = model.predict(X_test).flatten()

25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 220ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


In [ ]:
pred = np.concatenate([train_pred, val_pred, test_pred])
df_pred = pd.DataFrame(data["temp_max"].copy())
df_pred.columns = ["actual"]
df_pred = df_pred[WINDOW:]
df_pred["predicted"] = pred

df_long = df_pred[800:].reset_index()
df_long = df_long.melt(id_vars='index', value_vars=['actual', 'predicted'],
                        var_name='Type', value_name='Temperature')

fig_val = px.line(df_long, x='index', y='Temperature', color='Type',
                  title="Validation Results",
                  labels={'Temperature': 'Temperature', 'index': 'Time'},
                  color_discrete_map={'actual': 'blue', 'predicted': 'red'})
fig.update_layout(width=700, height=500)

fig_val.show()

In [ ]:
fig_test = px.line(df_pred[1000:], title="Test Results",
                   labels={'value': 'Temperature', 'index': 'Time'},
                   line_shape='linear')
fig_test.update_traces(name='Actual', line_color='blue')
fig_test.add_scatter(x=df_pred.index[1000:], y=df_pred['predicted'][1000:],
                      mode='lines', name='Predicted', line=dict(color='red'))
fig.update_layout(width=700, height=500)
fig_test.show()